# Dish ontology

In [1]:
from owlready2 import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## Create ontology

In [2]:
dish_ontology = get_ontology('http://test.org/dish.owl')

## Main atomic concepts and their properties

In [3]:
with dish_ontology:
    class Dish(Thing): pass

    class is_sweet(Dish >> bool, FunctionalProperty): pass
    class is_prepared_for(Dish >> int, FunctionalProperty): pass
    class is_cooked(Dish >> bool, FunctionalProperty): pass
    class is_liquid(Dish >> bool, FunctionalProperty): pass
    
    class PlaceOfOrigin(Thing): pass
    
    class is_in_region(PlaceOfOrigin >> PlaceOfOrigin, TransitiveProperty, ReflexiveProperty): pass
    class has_subregion(PlaceOfOrigin >> PlaceOfOrigin, TransitiveProperty, ReflexiveProperty):
        inverse_property = is_in_region
        pass
    
    class is_dish_in(Dish >> PlaceOfOrigin): pass
    class has_dish(PlaceOfOrigin >> Dish):
        inverse_property = is_dish_in
        pass
    
    class is_dish_in_region(Dish >> PlaceOfOrigin):
        equivalent_to = PropertyChain([is_dish_in, is_in_region])
        pass
    
    class Ingredient(Thing): pass
    
    class has_gluten(Ingredient >> bool, FunctionalProperty): pass
    
    class has_ingredient(Dish >> Ingredient): pass
    class is_ingredient_in(Ingredient >> Dish):
        inverse_property = has_ingredient
        pass

## Composed concepts and their properties

- subclasses of `Ingredient`

In [4]:
with dish_ontology:
    class GlutenFree(Ingredient):
        is_a = [has_gluten.value(False)]
        pass
    
    class Meat(GlutenFree): pass

    class is_raw(Meat >> bool, FunctionalProperty): pass
    
    class Chicken(Meat): pass
    class Beef(Meat): pass
    class Pork(Meat): pass
    
    class Fish(GlutenFree): pass
    
    class Vegetable(GlutenFree): pass
    
    class Dairy(GlutenFree): pass
    
    class Egg(GlutenFree): pass
    
    class Flour(Ingredient): pass

    class Fruit(GlutenFree): pass

- subclasses of `Dish`

In [5]:
with dish_ontology:
    class Salad(Dish):
        is_a = [is_cooked.value(False) & is_sweet.value(False) & is_prepared_for.max(20)]
        pass

    class MainDish(Dish):
        is_a = [is_cooked.value(True) & is_sweet.value(False)]
        pass
    
    class Soup(Dish):
        is_a = [is_liquid.value(True)]
        pass
    
    class is_pureed(Soup >> bool, FunctionalProperty): pass

    class CreamSoup(Soup):
        is_a = [is_pureed.value(True)]
        pass
    
    class Dessert(Dish):
        is_a = [is_sweet.value(True)]
        pass

    class Cake(Dessert):
        is_a = [has_ingredient.some(Flour)]
        pass

    class FruitCake(Cake):
        is_a = [has_ingredient.some(Fruit)]
        pass
    
    class VegetarianDish(Dish):
        equivalent_to = [Not(has_ingredient.some(Meat)) & Not(has_ingredient.some(Fish))]
        pass
    
    class VeganDish(VegetarianDish):
        equivalent_to = [Not(has_ingredient.some(Dairy)) & Not(has_ingredient.some(Egg))]
        pass

- subclasses of `Dish` based on `PlaceOfOrigin` instances 

In [6]:
Bulgaria = PlaceOfOrigin('Bulgaria')
Mexico = PlaceOfOrigin('Mexico')
Asia = PlaceOfOrigin('Asia')
Japan = PlaceOfOrigin('Japan', is_in_region = [Asia])

with dish_ontology:
    class BulgarianDish(Dish):
        equivalent_to = [is_dish_in_region.value(Bulgaria)]
        pass
    
    class JapaneseDish(Dish):
        equivalent_to = [is_dish_in_region.value(Japan)]
        pass
    
    class AsianDish(Dish):
        equivalent_to = [is_dish_in_region.value(Asia)]
        pass
    
    class MexicanDish(Dish):
        equivalent_to = [is_dish_in_region.value(Mexico)]
        pass

## Instances

In [7]:
chicken_wing = Chicken('chicken_wing')
chicken_leg = Chicken('chicken_leg')
chicken_fillet = Chicken('chicken_fillet')
whole_chicken = Chicken('whole_chicken')

beef_steak = Beef('beef_steak')

bacon = Pork('bacon')
pork_ribs = Pork('pork_ribs')
pork_steak = Pork('pork_steak')

salmon = Fish('salmon')

tomato = Vegetable('tomato')
cucumber = Vegetable('cucumber')
pepper = Vegetable('pepper')
carrot = Vegetable('carrot')
potato = Vegetable('potato')
onion = Vegetable('onion')
avocado = Vegetable('avocado')
broccoli = Vegetable('broccoli')
zucchini = Vegetable('zucchini')

cow_milk = Dairy('cow_milk')
cheese = Dairy('cheese')
yogurt = Dairy('jogurt')
butter = Dairy('butter')

chicken_egg = Egg('chicken_egg')
quail_egg = Egg('quail_egg')

all_purpose_flour = Flour('all_purpose_flour', has_gluten = True)
gluten_free_flour = Flour('gluten_free_flour', has_gluten = False)

apple = Fruit('apple')
plum = Fruit('plum')
pear = Fruit('pear')
banana = Fruit('banana')
pumpkin = Fruit('pumpkin')

baking_powder = GlutenFree('baking_powder')
cocoa_powder = GlutenFree('cocoa_powder')
vanilla_extract = GlutenFree('vanilla_extract')
white_sugar = GlutenFree('white_sugar')
brown_sugar = GlutenFree('brown_sugar')
cinnamon = GlutenFree('cinnamon')

ginger = GlutenFree('ginger')
pumpkin_seeds = GlutenFree('pumpkin_seeds')
salt = GlutenFree('salt')
black_pepper = GlutenFree('black_pepper')
vegetable_oil = GlutenFree('vegetable_oil')
water = GlutenFree('water')
rice = GlutenFree('rice')

brownies = Dessert('brownies', is_cooked = True, is_prepared_for = 60, 
                   has_ingredient = [all_purpose_flour, baking_powder, butter, cocoa_powder, vanilla_extract, white_sugar])
apple_cinnamon_muffin = FruitCake('apple_cinnamon_muffin', is_cooked = True, is_prepared_for = 45, 
                                 has_ingredient = [apple, gluten_free_flour, brown_sugar, cinnamon])
apple_cinnamon_muffin.is_a.append(VeganDish)

beef_stew = MainDish('beef_stew', is_prepared_for = 140,
                    has_ingredient = [beef_steak, black_pepper, carrot, onion, potato, vegetable_oil, water])

bacon_with_eggs = MainDish('bacon_with_eggs', is_prepared_for = 15, has_ingredient = [bacon, chicken_egg, vegetable_oil])

chicken_fillet_with_vegetables = MainDish('chicken_fillet_with_vegetables', is_prepared_for = 15,
                                          has_ingredient = [chicken_fillet, pepper, onion, zucchini, broccoli, salt, black_pepper, vegetable_oil])

sushi = MainDish('sushi', is_prepared_for = 60, is_dish_in = [Japan], has_ingredient = [rice, avocado, salmon])

shopska_salad = Salad('shopska_salad', is_prepared_for = 20, has_ingredient = [cucumber, tomato, pepper, cheese, onion])
shopska_salad.is_a.append(BulgarianDish)

pumpkin_cream_soup = CreamSoup('pumpkin_cream_soup', is_prepared_for = 60, has_ingredient = [pumpkin, pumpkin_seeds, ginger, water])
pumpkin_cream_soup.is_a.append(VeganDish)

## Save the ontology

In [8]:
dish_ontology.save('./dish_ontology.owl')

## Reasoning

In [9]:
def reasoner():
    owlready2.JAVA_EXE = "C:\\Program Files\\java\\jdk-13\\bin\\java.exe"

    with dish_ontology:
        sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)

reasoner()

* Owlready2 * Running Pellet...
    C:\Program Files\java\jdk-13\bin\java.exe -Xmx2000M -cp C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Users\kikir\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Users\kikir\AppData\Loc

* Owlready * Adding relation dish.cheese is_in_region dish.cheese
* Owlready * Adding relation dish.cheese has_subregion dish.cheese
* Owlready * Adding relation dish.avocado is_in_region dish.avocado
* Owlready * Adding relation dish.avocado has_subregion dish.avocado
* Owlready * Adding relation dish.pumpkin is_in_region dish.pumpkin
* Owlready * Adding relation dish.pumpkin has_subregion dish.pumpkin
* Owlready * Adding relation dish.plum is_in_region dish.plum
* Owlready * Adding relation dish.plum has_subregion dish.plum
* Owlready * Adding relation dish.cocoa_powder is_in_region dish.cocoa_powder
* Owlready * Adding relation dish.cocoa_powder has_subregion dish.cocoa_powder
* Owlready * Adding relation dish.Mexico is_in_region dish.Mexico
* Owlready * Adding relation dish.Mexico has_subregion dish.Mexico
* Owlready * Adding relation dish.broccoli is_in_region dish.broccoli
* Owlready * Adding relation dish.broccoli has_subregion dish.broccoli
* Owlready * Adding relation dish.pum

* Owlready2 * Pellet took 2.349466562271118 seconds
* Owlready * Equivalenting: owl.Thing dish.Dish
* Owlready * Equivalenting: owl.Thing dish.PlaceOfOrigin
* Owlready * Equivalenting: dish.Dish owl.Thing
* Owlready * Equivalenting: dish.Dish dish.PlaceOfOrigin
* Owlready * Equivalenting: dish.PlaceOfOrigin owl.Thing
* Owlready * Equivalenting: dish.PlaceOfOrigin dish.Dish
* Owlready * Reparenting dish.Mexico: {dish.PlaceOfOrigin} => {dish.Dish, dish.PlaceOfOrigin}
* Owlready * Reparenting dish.Bulgaria: {dish.PlaceOfOrigin} => {dish.Dish, dish.PlaceOfOrigin}
* Owlready * Reparenting dish.Japan: {dish.PlaceOfOrigin} => {dish.Dish, dish.PlaceOfOrigin}
* Owlready * Reparenting dish.Asia: {dish.PlaceOfOrigin} => {dish.Dish, dish.PlaceOfOrigin}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


## Some tests

In [10]:
print(sushi.is_a)
print(sushi.is_dish_in)
print(sushi.get_properties())
print(sushi.INDIRECT_get_properties())
print(sushi.is_instance_of)
print(sushi.__class__)

[dish.MainDish]
[dish.Japan]
{dish.is_sweet, dish.is_dish_in, dish.is_in_region, dish.is_cooked, dish.has_ingredient, dish.has_subregion, dish.is_prepared_for}
{dish.is_sweet, dish.is_dish_in, dish.is_in_region, dish.is_cooked, dish.has_ingredient, dish.has_subregion, dish.is_prepared_for}
[dish.MainDish]
dish.MainDish
